* https://www.kaggle.com/c/fake-news-pair-classification-challenge/data
* https://github.com/fxsjy/jieba

BERT
* https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb#scrollTo=Hcpfl4N2EdOk
* https://github.com/google-research/bert/blob/master/multilingual.md

Gluon-bert
* https://gluon-nlp.mxnet.io/examples/sentence_embedding/bert.html

Embedding options
* https://github.com/Embedding/Chinese-Word-Vectors (Embedding 1)
* https://github.com/facebookresearch/fastText/blob/master/docs/crawl-vectors.md (Embedding 2)

Performance
* Using Embedding 1: Performance 0.70640
* Using Embedding 2: Performance 0.68641
* Using BERT: Performance 0.849
* Using BERT_WEIGHTED: Performance 0.85526
* Using BERT_WEIGHTED+SameTID-Agreed: 0.85479
    * Example: df_test[df_test.pred_label != df_test.pred_label2]인 아이들 다 agreed
* Using BERT_WEIGHTED_train100: Performance 0.85634
* Using BERT_ensemble_3models (cn, multi, uncased24): Performance 0.85619
내일 submission: train 전체로 그냥 돌린거 (3에폭), 여러모델 최종값 merge - 어차피 eval으로 뭘 하는 게 아니니까.

In [1]:
cd /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/bert_repo

/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/bert_repo


In [2]:
import modeling
import optimization
import run_classifier
import tokenization

In [3]:
cd /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook

/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook


In [4]:
import datetime
import glob
import pandas as pd
import numpy as np
import os
import pickle

In [5]:
import tensorflow as tf
import keras

os.environ["CUDA_VISIBLE_DEVICES"] = "3,4,5"

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17237520278268438755
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12639676126145614516
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6927687444929730414
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15580053504
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5579352009766880559
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:0b:00.0, compute capability: 7.0"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 14546052711
locality {
  bus_id: 2
  numa_node: 1
  links {
    link {
      device_id: 2
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 16600438341523242333
physical_device_desc: "device: 1, name: Tesla V100-S

In [7]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    all_label_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.segment_ids)
        all_label_ids.append(feature.label_id)

    def input_fn(params):

        batch_size = 2

        num_examples = len(features)

        # This is for demo purposes and does NOT scale to large data sets. We do
        # not use Dataset.from_generator() because that uses tf.py_func which is
        # not TPU compatible. The right way to load data is with TFRecordReader.
        d = tf.data.Dataset.from_tensor_slices({
            "input_ids":
                tf.constant(
                    all_input_ids, shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "input_mask":
                tf.constant(
                    all_input_mask,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "segment_ids":
                tf.constant(
                    all_segment_ids,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "label_ids":
                tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
        })

        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
        return d

    return input_fn

In [8]:
BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12', 'multi_cased_L-12_H-768_A-12', 'uncased_L-24_H-1024_A-16']
['/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_'+model for model in BERT_MODEL_LIST]

['/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_chinese_L-12_H-768_A-12',
 '/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_multi_cased_L-12_H-768_A-12',
 '/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_uncased_L-24_H-1024_A-16']

In [9]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model

exp_marker = 'crude-ensemble-geomean'
TASK_DATA_DIR = '../data/FakeNews_BERT/FakeNews_BERT/'


# BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12', 'multi_cased_L-12_H-768_A-12', 'uncased_L-24_H-1024_A-16']
BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12']

for BERT_MODEL in BERT_MODEL_LIST:
    BERT_PRETRAINED_DIR = '../data/' + BERT_MODEL
    OUTPUT_DIR = '/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_'+BERT_MODEL

    print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
    print('***** Model output directory: {} *****'.format(OUTPUT_DIR))
    
    if BERT_MODEL == 'uncased_L-24_H-1024_A-16':
        TRAIN_BATCH_SIZE = 12
        EVAL_BATCH_SIZE = 12
        TEST_BATCH_SIZE = 12
        LEARNING_RATE = 2e-5
        NUM_TRAIN_EPOCHS = 3.0
        WARMUP_PROPORTION = 0.1
        MAX_SEQ_LENGTH = 43  # According to our dataset, length of 43 can cover 99.5% of the titles.
        TASK = 'FakeEN' 
    else:
        # Model Hyper Parameters
        TRAIN_BATCH_SIZE = 128 # 32
        EVAL_BATCH_SIZE = 32  # 8    
        TEST_BATCH_SIZE = 32  # 8    
        LEARNING_RATE = 2e-5
        NUM_TRAIN_EPOCHS = 3.0
        WARMUP_PROPORTION = 0.1
        MAX_SEQ_LENGTH = 43  # According to our dataset, length of 43 can cover 99.5% of the titles.
        TASK = 'Fake'
        
    # Model configs
    SAVE_CHECKPOINTS_STEPS = 1000
    ITERATIONS_PER_LOOP = 1000
    NUM_TPU_CORES = 8
    VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
    CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
    INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
    DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

    processors = {
      "cola": run_classifier.ColaProcessor,
      "mnli": run_classifier.MnliProcessor,
      "mrpc": run_classifier.MrpcProcessor,
      "fake": run_classifier.FakeProcessor,
      "fakeen": run_classifier.FakeENProcessor,  
    }
    processor = processors[TASK.lower()]()
    label_list = processor.get_labels()
    
    tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
    tpu_cluster_resolver = None

    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        model_dir=OUTPUT_DIR,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=ITERATIONS_PER_LOOP,
            num_shards=NUM_TPU_CORES,
            per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

    train_examples = processor.get_train_examples(TASK_DATA_DIR)
    num_train_steps = int(
        len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    model_fn = run_classifier.model_fn_builder(
        bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
        num_labels=len(label_list),
        init_checkpoint=INIT_CHECKPOINT,
        learning_rate=LEARNING_RATE,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        use_tpu=False,
        use_one_hot_embeddings=True)

    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=False,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=TRAIN_BATCH_SIZE,
        eval_batch_size=EVAL_BATCH_SIZE)

    # Train the model.
    print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
    train_features = run_classifier.convert_examples_to_features(
        train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    print('***** Started training at {} *****'.format(datetime.datetime.now()))
    print('  Num examples = {}'.format(len(train_examples)))
    print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
    tf.logging.info("  Num steps = %d", num_train_steps)
    train_input_fn = run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    print('***** Finished training at {} *****'.format(datetime.datetime.now()))

    # Test the model.
    test_examples = processor.get_test_examples(TASK_DATA_DIR)
    test_features = run_classifier.convert_examples_to_features(
        test_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

    # Eval will be slightly WRONG on the TPU because it will truncate
    # the last batch.
    test_input_fn = input_fn_builder(          # run_classifier.
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=True)

    result = estimator.predict(input_fn=test_input_fn)

    raw_result_list = []
    result_list = []
    for rr in result:
        raw_result_list.append(rr)
        result_list.append(np.argmax(rr))

        
    test_path = '../data/test.csv'
    submission_path = '../data/sample_submission_BERT_'+BERT_MODEL+'.csv'
    submission_all_path = '../data/sample_submission_BERT_'+BERT_MODEL+'_ALL.csv'
    result_raw_path = '../data/sample_submission_sd_BERT_'+BERT_MODEL+'_RESULT_SOFTMAX.p'
    df_test = pd.read_csv(test_path)
    df_test['pred_label'] = result_list
    idx_label = {0:'unrelated', 1:'agreed', 2:'disagreed'}
    df_test['pred_label'] = df_test['pred_label'].apply(lambda x: idx_label[x])

    df_test[['id', 'pred_label']].rename(columns={'id':'Id','pred_label':'Category'}).to_csv(submission_path, index=False)
    df_test.to_csv(submission_all_path, index=False)

    pickle.dump(raw_result_list, open(result_raw_path, 'wb'))

***** BERT pretrained directory: ../data/chinese_L-12_H-768_A-12 *****
***** Model output directory: /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_chinese_L-12_H-768_A-12 *****
INFO:tensorflow:Using config: {'_model_dir': '/home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_chinese_L-12_H-768_A-12', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1a7827e160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', 

INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tenso

INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT

INFO:tensorflow:  name = bert/encoder/layer_9/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/enc

INFO:tensorflow:Saving checkpoints for 4000 into /home/dmlab/sundong/competition/wsdm2019/fakenews/notebook/results_chinese_L-12_H-768_A-12/model.ckpt.
INFO:tensorflow:global_step/sec: 2.3862
INFO:tensorflow:examples/sec: 305.434
INFO:tensorflow:global_step/sec: 2.59689
INFO:tensorflow:examples/sec: 332.402
INFO:tensorflow:global_step/sec: 2.59723
INFO:tensorflow:examples/sec: 332.446
INFO:tensorflow:global_step/sec: 2.59412
INFO:tensorflow:examples/sec: 332.047
INFO:tensorflow:global_step/sec: 2.59675
INFO:tensorflow:examples/sec: 332.385
INFO:tensorflow:global_step/sec: 2.59583
INFO:tensorflow:examples/sec: 332.266
INFO:tensorflow:global_step/sec: 2.59365
INFO:tensorflow:examples/sec: 331.988
INFO:tensorflow:global_step/sec: 2.59581
INFO:tensorflow:examples/sec: 332.263
INFO:tensorflow:global_step/sec: 2.59367
INFO:tensorflow:examples/sec: 331.99
INFO:tensorflow:global_step/sec: 2.5951
INFO:tensorflow:examples/sec: 332.173
INFO:tensorflow:Saving checkpoints for 5000 into /home/dmlab/

...

INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/dense/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/dense/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/LayerNorm/beta:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/LayerNorm/gamma:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/intermediate/dense/kernel:0, shape = (1024, 4096), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/intermediate/dense/bias:0, shape = (4096,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/output/dense/kernel:0, shape = (4096, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/output/dense/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/output/LayerNorm/beta:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:

INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/query/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/key/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/key/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/value/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/value/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/output/dense/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/output/dense/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/output/LayerNorm/beta:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/output/LayerNorm/gamma:0, shape = (1024,), *I

INFO:tensorflow:  name = bert/encoder/layer_10/intermediate/dense/bias:0, shape = (4096,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/dense/kernel:0, shape = (4096, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/dense/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/LayerNorm/beta:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/LayerNorm/gamma:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/query/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/query/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/key/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/key/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  

INFO:tensorflow:  name = bert/encoder/layer_15/attention/self/value/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_15/attention/output/dense/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_15/attention/output/dense/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_15/attention/output/LayerNorm/beta:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_15/attention/output/LayerNorm/gamma:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_15/intermediate/dense/kernel:0, shape = (1024, 4096), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_15/intermediate/dense/bias:0, shape = (4096,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_15/output/dense/kernel:0, shape = (4096, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_15/output/dense/bias:0, shape = (1024,), *INIT_FRO

INFO:tensorflow:  name = bert/encoder/layer_19/output/LayerNorm/gamma:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_20/attention/self/query/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_20/attention/self/query/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_20/attention/self/key/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_20/attention/self/key/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_20/attention/self/value/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_20/attention/self/value/bias:0, shape = (1024,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_20/attention/output/dense/kernel:0, shape = (1024, 1024), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_20/attention/output/dense/bias:0, shape = (1024,), *I

In [45]:
## Geomean

submission_ensemble_path = '../data/sample_submission_sd_BERT_weightedavg_4model_cn.csv'
smax = {}
BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12', 'chinese_L-12_H-768_A-12_EN', 'multi_cased_L-12_H-768_A-12', 'uncased_L-24_H-1024_A-16']
# BERT_MODEL_LIST = ['chinese_L-12_H-768_A-12']
for BERT_MODEL in BERT_MODEL_LIST: 
    pkls_path = '../data/sample_submission_sd_BERT_'+BERT_MODEL+'_RESULT_SOFTMAX.p'
    with open(pkls_path, 'rb') as file:
        smax[BERT_MODEL] = np.asarray(pickle.load(file))
        
# ens_rslt = np.multiply(smax['chinese_L-12_H-768_A-12', smax['multi_cased_L-12_H-768_A-12'], smax['uncased_L-24_H-1024_A-16']])   
ens_rslt = np.average(np.array([smax['chinese_L-12_H-768_A-12'], smax['multi_cased_L-12_H-768_A-12'], smax['uncased_L-24_H-1024_A-16'], smax['chinese_L-12_H-768_A-12_EN']]), 
                      axis=0, weights=[0.6,0.1,0.1,0.2])


df_test = pd.read_csv(test_path)
df_test['pred_label'] = np.argmax(ens_rslt, axis=1)
idx_label = {0:'unrelated', 1:'agreed', 2:'disagreed'}
df_test['pred_label'] = df_test['pred_label'].apply(lambda x: idx_label[x])

df_test[['id', 'pred_label']].rename(columns={'id':'Id','pred_label':'Category'}).to_csv(submission_ensemble_path, index=False)

In [44]:
ens_rslt = np.average(np.array([smax['chinese_L-12_H-768_A-12'], smax['multi_cased_L-12_H-768_A-12'], smax['uncased_L-24_H-1024_A-16'], smax['chinese_L-12_H-768_A-12_EN']]), 
                      axis=0, weights=[0.6,0.1,0.1,0.2])

In [49]:
ens_rslt

array([[  9.93777174e-01,   2.54625798e-04,   5.96824945e-03],
       [  9.85447580e-01,   8.00269205e-04,   1.37521305e-02],
       [  9.86330760e-01,   7.39814398e-03,   6.27108336e-03],
       ..., 
       [  9.06974125e-01,   8.91996742e-02,   3.82621692e-03],
       [  8.94026029e-01,   1.03784758e-01,   2.18923882e-03],
       [  7.20862854e-01,   2.77621056e-01,   1.51606923e-03]])

In [46]:
df_test['pred_label'].value_counts()

unrelated    52573
agreed       25791
disagreed     1762
Name: pred_label, dtype: int64

In [47]:
df_train = pd.read_csv('../data/train.csv')
dft_tr = df_train['label'].value_counts()
dft_tr/np.sum(dft_tr)

unrelated    0.684173
agreed       0.290040
disagreed    0.025787
Name: label, dtype: float64

In [48]:
dft = df_test['pred_label'].value_counts()
dft/np.sum(dft)

unrelated    0.656129
agreed       0.321881
disagreed    0.021990
Name: pred_label, dtype: float64